## Merging Cities Data Set with Elected Officials Data Set for Initial Visual

This notebook merges **uscities.csv** and **elof_cities.csv** to plot the data from the elected officials data set onto the U.S. map. The data set, **uscities.csv**, contains latitude/longitude coordinates of cities in the U.S., which are needed for this particular graphing method. This data set was found at **https://simplemaps.com/data/us-cities**.



In [3]:
# Import Statements

!pip install plotly

import pandas as pd
import numpy as np
from scipy import stats
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

In [4]:
# Reads in the cities dataset
df_cities = pd.read_csv('uscities.csv')

# Converts all variables to lowercase
df_cities.columns = df_cities.columns.str.replace('\s+', '_').str.lower()

# Displays header
df_cities.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276.0,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574.0,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019.0,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526.0,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440


In [5]:
# Keeps only the necessary columns
df_cities_only = df_cities[['city', 'lat', 'lng', 'population', 'state_id']]

# Renames the state column for merging purposes
df_cities_only = df_cities_only.rename(columns = {'state_id': 'state'})
df_cities_only

,city,lat,lng,population,state
0,New York,40.6943,-73.9249,18713220,NY
1,Los Angeles,34.1139,-118.4068,12750807,CA
2,Chicago,41.8373,-87.6862,8604203,IL
3,Miami,25.7839,-80.2102,6445545,FL
4,Dallas,32.7936,-96.7662,5743938,TX
...,...,...,...,...,...
28367,Poso Park,35.8114,-118.6364,2,CA
28368,Lotsee,36.1334,-96.2091,2,OK
28369,The Ranch,47.3198,-95.6952,2,MN
28370,Roswell,44.0060,-97.6975,2,SD


In [6]:
# Read in the elof cities dataset
df_elof_cities = pd.read_csv('elof_cities.csv')

# Converts all variables to lowercase
df_elof_cities.columns = df_elof_cities.columns.str.replace('\s+', '_').str.lower()

# Displays header
df_elof_cities.head()

FileNotFoundError: [Errno 2] No such file or directory: 'elof_cities.csv'

In [ ]:
# Checks for any duplicate cities
df_elof_cities[df_elof_cities.city.duplicated()]

In [ ]:
# Since San Diego and New York are the only 
# duplicated cities, we will removes these
# rows and add the count to their original row
df_elof_cities = df_elof_cities.drop_duplicates(subset = ['city'])
df_elof_cities.city

# Note that we are losing a count of 2 for each state when
# we do this. This would be a fix we would implement in the future.

In [ ]:
# Merge the two DataFrames
merged_cities = df_elof_cities.merge(df_cities_only, how='inner')

# Save new merged DataFrame as a csv
merged_cities.to_csv('merged_cities.csv')

merged_cities.sort_values('count')

In [ ]:
# Reviewing the new city list
# Only has 93 items - another fix for the future
# Text most likeley did not line up exactly for each column 

city_list = []
for item in merged_cities.city:
    city_list.append(item)
    
len(city_list)

## Elected Officials Data Set Map Visualization

The outline of this code was originally taken from an example on the plotly website: https://plotly.com/python/bubble-maps/. The code was modified slightly to match our dataframe / data set.

In [ ]:
# Used the example from https://plotly.com/python/bubble-maps/ to format this

merged_cities['text'] = (merged_cities['city']
                         + '<br>Elected Officials Count: ' + merged_cities['count'].astype(str)
                         + '<br>Population: ' + (round(merged_cities['population']/1e6, 2)).astype(str)+' million')

# 'Buckets' used to separate elected officials counts by city
# These were based on percentiles that the data fell into
limits = [(5,7),(8,9),(10,13),(14,56)]

# Associated colors with the buckets from above
colors = ['royalblue','crimson','lightseagreen','orange']

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    merged_cities_sub = merged_cities[(merged_cities['count'] >= lim[0])
                                      & (merged_cities['count'] <= lim[1])]

    # Plotting on top of the current fig
    # for each loop
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = merged_cities_sub['lng'],
        lat = merged_cities_sub['lat'],
        text = merged_cities_sub['text'],
        marker = dict(size = merged_cities_sub['count']*10,
                      color = colors[i],
                      line_color='rgb(40,40,40)',
                      line_width=0.5,
                      sizemode = 'area'),
        name = '{0} - {1}'.format(lim[0],lim[1])))


fig.update_layout(
        # title_text = '2019 Elected Officials Data Count',
        showlegend = True,
        margin={"r":0,"t":0,"l":0,"b":0}, #adjust window size
        geo = dict(scope = 'usa',
                   landcolor = 'rgb(217, 217, 217)')
    )

fig.show()